1. <a href="#load"> Loading Data </a>
2. <a href="#prep"> Pre-processing </a>
3. <a href="#bmod"> Build Model </a>
4. <a href="#eval"> Evaluate Model </a>
5. <a href="#pred"> Predict </a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# <a id="load">Loading Data</a>

CIFAR-10 is a dataset of 50,000 32x32 color training images, labeled over 10 classes

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:

fig = plt.figure(figsize=(8, 8))
columns = 6
rows = 4
for i in range(1, columns*rows +1):
    img = x_train[i]
    fig.add_subplot(rows, columns, i)
    plt.axis('off')
    plt.imshow(img)
plt.show()

# <a id="prep"> Pre-processing </a>

In [ ]:
print(x_train.max())
print(x_test.max())

In [ ]:
x_train = x_train/225
x_test = x_test/255

In [ ]:
y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test,10)

# <a id="bmod"> Build Model </a>

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(32, 32, 3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(4,4), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
model.fit(x_train,
          y_cat_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_test,y_cat_test),
          callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
losses[['accuracy','val_accuracy']].plot()

# <a id="eval"> Evaluate Model </a>

In [ ]:
print(model.metrics_names)
print(model.evaluate(x_test,y_cat_test,verbose=0))

In [ ]:
predictions = np.argmax(model.predict(x_test), axis=-1)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
confusion_matrix(y_test,predictions)

# <a id="pred"> Predict </a>

In [ ]:
classes = [0,1,2,3,4,5,6,7,8,9]
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# https://www.cs.toronto.edu/~kriz/cifar.html

d = dict(zip(classes, class_names))

In [ ]:
my_image = x_test[20]

plt.imshow(my_image)

In [ ]:
input_img = my_image.reshape(1,32,32,3)

predictions = np.argmax(model.predict(input_img), axis=-1)[0]

print(f"True class: {d[y_test[20][0]]} \n\nPredicted class: {d[predictions]}")